In [ ]:
import serial
import json
import numpy as np
import time

ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=1)

ACTIONS = [0, 1, 2]   # forward, left, right

def get_state():
    line = ser.readline().decode().strip()
    if not line:
        return None
    try:
        data = json.loads(line)
        return np.array([data["L"], data["C"], data["R"]])
    except:
        return None


def send_action(a):
    ser.write(str(a).encode())


def reward_function(state):
    L, C, R = state

    # premio por avanzar sin obstaculos
    if C == 0:
        return 1.0

    # penalizar si hay obstáculo de frente
    if C == 1:
        return -5.0

    return -0.1


# -------------------------
#  LOOP RL BÁSICO
# -------------------------
from random import choice

Q = np.zeros((8, 3))  # 8 estados, 3 acciones
alpha = 0.3
gamma = 0.9
epsilon = 0.2

def state_to_index(s):
    return s[0]*4 + s[1]*2 + s[2]*1

while True:
    state = get_state()
    if state is None:
        continue

    idx = state_to_index(state)

    # política epsilon-greedy
    if np.random.rand() < epsilon:
        action = choice(ACTIONS)
    else:
        action = np.argmax(Q[idx])

    send_action(action)

    time.sleep(0.15)

    new_state = get_state()
    if new_state is None:
        continue

    reward = reward_function(new_state)

    new_idx = state_to_index(new_state)

    # Q-learning
    Q[idx, action] = Q[idx, action] + alpha * (
        reward + gamma * np.max(Q[new_idx]) - Q[idx, action]
    )

    print("State:", state, "Action:", action, "Reward:", reward)


SerialException: [Errno 2] could not open port /dev/ttyUSB1: [Errno 2] No such file or directory: '/dev/ttyUSB1'